In [3]:
%%writefile diet_recommendation_app.py
import streamlit as st
import pandas as pd
import re
import json
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Function definitions
def scaling(dataframe):
    scaler = StandardScaler()
    prep_data = scaler.fit_transform(dataframe.iloc[:, 6:15].to_numpy())
    return prep_data, scaler

def nn_predictor(prep_data):
    neigh = NearestNeighbors(metric='cosine', algorithm='brute')
    neigh.fit(prep_data)
    return neigh

def filter_recipes(df, bmi, goal, diet_preference, exclude_ingredients=[]):
    non_veg_keywords = ['chicken', 'beef', 'fish', 'pork', 'egg', 'shrimp', 'lobster','lamb']
    veg_keywords = ['tofu', 'lentil', 'vegetable', 'beans', 'quinoa']

    recommended_recipes = pd.DataFrame()
    if bmi < 18.5:
        recommended_recipes = df[(df['Calories'] > 300) & (df['FiberContent'] > 5)]
        action = "You're underweight. Don't worry, we’ll get into shape in no time! This recipe is rich in calories and protein to help you gain healthy weight."
    elif 18.5 <= bmi <= 24.9:
        recommended_recipes = df[(df['Calories'] < 400) & (df['FiberContent'] > 4)]
        action = "You're at a healthy weight! Keep it up! This recipe provides balanced nutrition to help maintain your fitness level."
    else:
        recommended_recipes = df[(df['Calories'] < 250) & (df['FiberContent'] > 5)]
        action = "You're overweight. No worries, we're on the right path! This recipe is designed to be low in calories and high in fiber to support your weight loss journey."
    if goal == 'muscle gain':
        recommended_recipes = recommended_recipes[(recommended_recipes['ProteinContent'] > 20) & (recommended_recipes['Calories'] > 400)]

    if diet_preference == 'veg':
        recommended_recipes = recommended_recipes[recommended_recipes['RecipeIngredientParts'].str.contains('|'.join(veg_keywords), case=False)]
    elif diet_preference == 'non-veg':
        recommended_recipes = recommended_recipes[recommended_recipes['RecipeIngredientParts'].str.contains('|'.join(non_veg_keywords), case=False)]

    if exclude_ingredients:
        exclude_pattern = '|'.join(exclude_ingredients)
        recommended_recipes = recommended_recipes[~recommended_recipes['RecipeIngredientParts'].str.contains(exclude_pattern, case=False)]

    return recommended_recipes, action

def plot_nutrient_chart(recipe):
    labels = ['Calories', 'Protein', 'Carbohydrates', 'Fiber']
    values = [recipe.Calories, recipe.ProteinContent, recipe.CarbohydrateContent, recipe.FiberContent]
    colors = ['lightcyan', 'lightblue', 'lightgreen', 'lightpink']
    
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, marker=dict(colors=colors))])
    fig.update_layout(title=f'Nutrient Breakdown for {recipe.Name}')
    return fig

def save_recipes_to_txt(recommended_recipes):
    file_path = "recommended_recipes.txt"
    with open(file_path, "w") as f:
        f.write("Top 5 Recommended Recipes\n\n")
        for idx, recipe in enumerate(recommended_recipes.head(5).itertuples(), 1):
            f.write(f"Recipe #{idx}: {recipe.Name}\n")
            f.write(f"Calories: {recipe.Calories} kcal\n")
            f.write(f"Protein: {recipe.ProteinContent}g\n")
            f.write(f"Carbs: {recipe.CarbohydrateContent}g\n")
            f.write(f"Fiber: {recipe.FiberContent}g\n\n")
            
            f.write("Ingredients:\n")
            ingredients = re.findall(r'"([^"]*)"', recipe.RecipeIngredientParts)
            for ingredient in ingredients:
                f.write(f"- {ingredient}\n")
            
            f.write("\nInstructions:\n")
            steps = re.split(r'(?<!\w),(?!\w)', recipe.RecipeInstructions)
            cleaned_steps = [step.replace('c(\"', '').replace('\"', '').replace('\")', '').strip() for step in steps if step.strip()]
            for i, step in enumerate(cleaned_steps, 1):
                f.write(f"Step {i}: {step}\n")
            f.write("\n---\n\n")
    return file_path
def main():
    st.title("Personalized Diet Recommendation System")
    
    age = int(st.number_input("Enter your age", min_value=1, step=1))
    height = float(st.number_input("Enter your height (in cm)", min_value=50.0, step=0.1))
    weight = float(st.number_input("Enter your weight (in kg)", min_value=30.0, step=0.1))
    goal = st.selectbox("Select your goal", ["weight loss", "muscle gain", "maintenance", "weight gain"])
    diet_preference = st.selectbox("Select your diet preference", ["veg", "non-veg", "mixed"])
    exclude_ingredients = [ingredient.strip() for ingredient in st.text_area("Enter ingredients to exclude (comma-separated)", "").split(',') if ingredient.strip()]

    if st.button("Get Recommendations"):
        try:
            df = pd.read_csv(r'C:\\Users\\Lavanyabh\\Desktop\\Sem3\\project\\dataset.csv', compression='gzip')
        except FileNotFoundError:
            st.error("Dataset not found. Please check the file path.")
            return

        bmi = weight / ((height / 100) ** 2)
        recommended_recipes, action = filter_recipes(df, bmi, goal, diet_preference, exclude_ingredients)

        st.markdown(f"<h2 style='color: #FF6347; font-weight: bold;'>Your BMI is: {bmi:.2f}</h2>", unsafe_allow_html=True)
        st.markdown(f"<h3 style='color: #32CD32; font-size: 18px;'>{action}</h3>", unsafe_allow_html=True)

        if not recommended_recipes.empty:
            st.write("### Top 5 Recommended Recipes")
            for idx, recipe in enumerate(recommended_recipes.head(5).itertuples(), 1):
                st.write(f"## Recipe #{idx}: {recipe.Name}")
                st.write(f"- **Calories:** {recipe.Calories} kcal")
                st.write(f"- **Protein:** {recipe.ProteinContent}g")
                st.write(f"- **Carbs:** {recipe.CarbohydrateContent}g")
                st.write(f"- **Fiber:** {recipe.FiberContent}g")
                
                with st.expander(f"Ingredients for {recipe.Name}"):
                    ingredients = re.findall(r'"([^"]*)"', recipe.RecipeIngredientParts)
                    for ingredient in ingredients:
                        st.write(f"  - {ingredient}")
                
                with st.expander(f"Instructions for {recipe.Name}"):
                    steps = re.split(r'(?<!\w),(?!\w)', recipe.RecipeInstructions)
                    cleaned_steps = [step.replace('c(\"', '').replace('\"', '').replace('\")', '').strip() for step in steps if step.strip()]
                    for i, step in enumerate(cleaned_steps, 1):
                        st.write(f"  **Step {i}:** {step}")
            
            best_recipe = recommended_recipes.iloc[0]
            st.write("## Best Recommended Recipe")
            st.write(f"### {best_recipe.Name}")
            st.plotly_chart(plot_nutrient_chart(best_recipe))
            txt_file = save_recipes_to_txt(recommended_recipes)
            with open(txt_file, "rb") as f:
                st.download_button(
                    label="Download Top 5 Recommended Recipes (TXT File)",
                    data=f,
                    file_name="recommended_recipes.txt",
                    mime="text/plain"
                )

if __name__ == "__main__":
    main()


Overwriting diet_recommendation_app.py


In [1]:
import os
print(os.getcwd())


C:\Users\Lavanyabh\Project_sem_3


In [ ]:
streamlit run diet_recommendation_app.py